In [1]:
from core.meta import meta_interface
%matplotlib inline

C:\Users\Kobel_K\.conda\envs\pipeline_b06\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
d = meta_interface(project_config_filepath="Z:Konstantin/project_config_server.yaml", project_name="analysis_01")

In [3]:
d.add_recording_config("Z:Konstantin/Calibration/220921/recording_config_210922.yaml")
d.add_recording_config("Z:Konstantin/Calibration/220922/recording_config_220922.yaml")
d.add_recording_config("Z:Konstantin/Calibration/220923/recording_config_230922.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/220927/recording_config_270922.yaml")
#some recordings that are unnecessary
#d.add_recording_config("Z:Konstantin/Calibration/221219/recording_config_191222.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/221220/recording_config_201222.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/221221/recording_config_211222.yaml")
#d.add_recording_config("Z:Konstantin/Calibration/221222/recording_config_221222.yaml")

In [4]:
d.initialize_meta_config()

In [5]:
d.create_recordings()

In [ ]:
d.load_meta_from_yaml(filepath="Z:Konstantin/analysis_01.yaml")

In [6]:
d.synchronize_recordings(verbose = True, test_mode = False)  # test_mode: bool, doesnt overwrite synchronized and analysed videos
# exclude recordingvideos based on synchronization

100%|███████████████████████████████████████████████████████████████████████████▉| 47400/47438 [09:57<00:00, 79.39it/s]


KeyboardInterrupt: 

In [ ]:
d.load_meta_from_yaml(filepath="test_data/My_project.yaml")

In [ ]:
d.create_calibrations()
# update fps, video_filepath, target_fps

In [ ]:
d.synchronize_calibrations(test_mode = True)  

# test_mode: bool, doesnt overwrite synchronized videos
# to speed up testing, remove some cams from valid_cam_IDs in project_config.yaml

# exclude calibrationvideos based on synchronization, change markerdetection_filepaths etc.

In [ ]:
d.calibrate()
# exclude calibrations based on calibration log and repeat calibration (not yet implemented) 

In [ ]:
d.triangulate_recordings()

In [ ]:
d.meta

In [ ]:
d.export_meta_to_yaml(filepath="test_data/My_project.yaml")

In [ ]:
d.load_meta_from_yaml(filepath="test_data/My_project.yaml")

In [ ]:
from core.triangulation_calibration_module import (
    Calibration,
    Triangulation_Positions,
    Triangulation_Recordings,
)

In [ ]:
a = Calibration(
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    project_config_filepath="test_data/project_config.yaml",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
a.run_synchronization()

In [ ]:
a.run_calibration(verbose=True)

In [ ]:
b = Triangulation_Recordings(
    recording_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/VGlut2-flp/September2022/196_F7-27/220826_OTT/",
)

In [ ]:
b.run_synchronization()

In [ ]:
b.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
c = Triangulation_Positions(
    positions_directory="test_data/Server_structure/Calibrations/220826/",
    calibration_directory="test_data/Server_structure/Calibrations/220826/",
    recording_config_filepath="test_data/Server_structure/Calibrations/220826/recording_config.yaml",
    project_config_filepath="test_data/project_config.yaml",
    output_directory="test_data/Server_structure/Calibrations/220826/",
)

In [ ]:
c.get_marker_predictions()

In [ ]:
c.run_triangulation(
    calibration_toml_filepath="test_data/Server_structure/Calibrations/220826/calibration_220826.toml"
)

In [ ]:
# special dependencies

In [ ]:
"""
pip install aniposelib
pip install ffmpeg
pip install ffmpeg-python
pip install imageio-ffmpeg
jupyter lab
Tkinter
"""